In [1]:
import pandas as pd

TRAIN_PATH = "/kaggle/input/nlp-getting-started/train.csv"
TEST_PATH  = "/kaggle/input/nlp-getting-started/test.csv"
SAMPLE_SUB_PATH = "/kaggle/input/nlp-getting-started/sample_submission.csv"

train = pd.read_csv(TRAIN_PATH)
test  = pd.read_csv(TEST_PATH)
sample = pd.read_csv(SAMPLE_SUB_PATH)

train.shape, test.shape, train.head(2)


((7613, 5),
 (3263, 4),
    id keyword location                                               text  \
 0   1     NaN      NaN  Our Deeds are the Reason of this #earthquake M...   
 1   4     NaN      NaN             Forest fire near La Ronge Sask. Canada   
 
    target  
 0       1  
 1       1  )

In [2]:
import re
import numpy as np

def clean_text(s: str) -> str:
    if pd.isna(s): return ""
    s = s.lower()
    s = re.sub(r"http\S+|www\.\S+", " URL ", s)
    s = re.sub(r"[^a-z0-9#@_'\s]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

train["text_clean"] = train["text"].map(clean_text)
test["text_clean"]  = test["text"].map(clean_text)
train[["text","text_clean","target"]].head(3)


,text,text_clean,target
0,Our Deeds are the Reason of this #earthquake M...,our deeds are the reason of this #earthquake m...,1
1,Forest fire near La Ronge Sask. Canada,forest fire near la ronge sask canada,1
2,All residents asked to 'shelter in place' are ...,all residents asked to 'shelter in place' are ...,1


In [3]:
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

X = train["text_clean"].values
y = train["target"].values

pipe = Pipeline([
    ("tfidf", TfidfVectorizer(
        ngram_range=(1,2),
        min_df=2,
        max_features=20000,
        sublinear_tf=True
    )),
    ("lr", LogisticRegression(
        solver="liblinear",
        C=4.0,
        class_weight="balanced",
        max_iter=200,
        random_state=42
    ))
])

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(pipe, X, y, cv=cv, scoring="f1")
print("CV F1 per fold:", scores)
print("CV F1 mean:", scores.mean(), "±", scores.std())


CV F1 per fold: [0.76502732 0.76108949 0.74299065 0.7511811  0.75700935]
CV F1 mean: 0.7554595837859999 ± 0.007738460875193669


In [4]:
pipe.fit(X, y)
pred = pipe.predict(test["text_clean"].values)

submission = pd.DataFrame({"id": test["id"], "target": pred.astype(int)})
submission_file = "/kaggle/working/submission_baseline.csv"
submission.to_csv(submission_file, index=False)
submission.head(), submission_file


(   id  target
 0   0       1
 1   2       1
 2   3       1
 3   9       0
 4  11       1,
 '/kaggle/working/submission_baseline.csv')

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session